In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy
from multiprocessing import Pool, Process

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep='\t', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,0
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,0
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.fillna(u'0', inplace=True)
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,
1,Halabardnik,115,115,10,5,6,5,2,3,0,
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [26]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.name = unit.name
        
        self.hp = unit.hp
        self.hp_left = self.hp
        
        self.speed = unit.speed
        self.attack = unit.attack
        self.defense = unit.defense
        self.dmg_min = unit.dmg_min
        self.dmg_max = unit.dmg_max
        self.shots = unit.shots
        self.attributes = unit.attributes      
        
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.hp)
            self.count -= num_killed + 1
            self.hp_left = self.hp - rem
            self.count = max(self.count, 0)
            
            
    def __calc_base_damage(self, other):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        att_to_def = self.attack - other.defense
        base_dmg_reduction = 0.
        base_dmg_bonus = 0.
        if att_to_def > 0:
            base_dmg_bonus = min(.05 * att_to_def, 3.)
        else:
            base_dmg_reduction = min(.025 * -att_to_def, .7)
            
        return base_dmg, base_dmg_bonus, base_dmg_reduction
    
            
    def attack_melee(self, other, dmg_bonus=0., melee_penalty=False):
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        if self.name == 'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if melee_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        
        
    def attack_range(self, other, dmg_bonus=0., range_penalty=False):
        assert self.shots > 0
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if range_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        self.shots -= 1

In [27]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [28]:
def fight(stackA, stackB, num_iter):    
    wins = {stackA.name : 0,
            stackB.name : 0}
    
    def units_order():
        temp = sorted([copy(stackA),copy(stackB)], key=lambda x: x.speed, reverse=True)
        if stackA.speed == stackB.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        melee_penalty_current = 'SHOOTING_ARMY' in current.attributes \
                                and not 'const_no_melee_penalty' in current.attributes
        current.attack_melee(other, melee_penalty=melee_penalty_current)
        
        if other.count > 0 and 'const_free_attack' not in current.attributes:
            melee_penalty_other = 'SHOOTING_ARMY' in other.attributes \
                                   and not 'const_no_melee_penalty' in other.attributes
            other.attack_melee(current, melee_penalty=melee_penalty_other)
            
        if current.count > 0 and 'const_two_attacks' in current.attributes \
                             and not 'SHOOTING_ARMY' in current.attributes:
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, distance):
        range_penalty = distance > 10 and current.name != 'Strzelec'
        current.attack_range(other, range_penalty=range_penalty)
        if 'const_two_attacks' in current.attributes and current.shots > 0:
            current.attack_range(other, range_penalty=range_penalty)
        return other, current
    
    starting_dist = 14
    if 'DOUBLE_WIDE' in stackA.attributes:
        starting_dist -= 1
    if 'DOUBLE_WIDE' in stackB.attributes:
        starting_dist -= 1
        

    for i in xrange(num_iter):
        current, other = units_order()
        
        if not 'SHOOTING_ARMY' in current.attributes and \
           not 'SHOOTING_ARMY' in other.attributes:
            while current.count > 0 and other.count > 0:
                current, other = melee_hit(current, other)
                
        elif current.shots > 0 and other.shots > 0:
            while current.count > 0 and current.shots > 0 and other.shots > 0:
                current, other = range_hit(current, other, starting_dist)
            if current.count > 0:
                ...
                
        else:
            distance = starting_dist
            ...
        
            
        winner = current if current.count > 0 else other
        wins[winner.name] += 1
        
    return wins

In [45]:
import time
t0 = time.time()
A = stack(make_unit(u'Pikinier'), 835)
B = stack(make_unit(u'Minotaur'), 94)

result = fight(A, B, 2000)
print result
time.time()-t0

{u'Minotaur': 908, u'Pikinier': 1092}


0.5028121471405029

In [46]:
result = fight(A, B, 2000)
print result

{u'Minotaur': 935, u'Pikinier': 1065}


In [67]:
A = stack(make_unit(u'Pikinier'), 5019)
B = stack(make_unit(u'Anioł'), 128)

result = fight(A, B, 2000)
result

Anioł: 128
Pikinier: 5019
----------------



{u'Anio\u0142': 1083, u'Pikinier': 917}

In [58]:
A = stack(make_unit(u'Anioł'), 1057)
B = stack(make_unit(u'Minotaur'), 5019)

result = fight(A, B, 2000)
result

Anioł: 1057
Minotaur: 5019
----------------



{u'Anio\u0142': 1022, u'Minotaur': 978}